# Clustering

## KMeans

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('clustering').getOrCreate()

In [2]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans

In [3]:
cluster_df = spark.read.csv("/clustering_dataset.csv", 
                            header=True, 
                            inferSchema=True)

In [4]:
cluster_df.show(75)

+----+----+----+
|col1|col2|col3|
+----+----+----+
|   7|   4|   1|
|   7|   7|   9|
|   7|   9|   6|
|   1|   6|   5|
|   6|   7|   7|
|   7|   9|   4|
|   7|  10|   6|
|   7|   8|   2|
|   8|   3|   8|
|   4|  10|   5|
|   7|   4|   5|
|   7|   8|   4|
|   2|   5|   1|
|   2|   6|   2|
|   2|   3|   8|
|   3|   9|   1|
|   4|   2|   9|
|   1|   7|   1|
|   6|   2|   3|
|   4|   1|   9|
|   4|   8|   5|
|   6|   6|   7|
|   4|   6|   2|
|   8|   1|   1|
|   7|   5|  10|
|  17|  25|  21|
|  15|  23|  32|
|  42|  25|  45|
|  41|  47|  21|
|  37|  20|  27|
|  40|  18|  26|
|  41|  28|  50|
|  32|  25|  40|
|  24|  29|  35|
|  47|  18|  47|
|  36|  42|  45|
|  49|  29|  15|
|  47|  39|  22|
|  38|  27|  25|
|  45|  23|  40|
|  23|  36|  19|
|  47|  40|  50|
|  37|  30|  40|
|  42|  48|  41|
|  29|  31|  21|
|  36|  39|  48|
|  50|  24|  31|
|  42|  44|  37|
|  37|  39|  46|
|  22|  40|  30|
|  17|  29|  41|
|  85| 100|  69|
|  68|  76|  67|
|  76|  70|  93|
|  62|  66|  91|
|  83|  93|  7

What you'll notice is, we'll notice that the data is grouped into three clusters. The first 25 rows or so all have values between one and 10. The second set of 25 rows all have data values between about 15 and 60. And then finally, the third set have data values between 60 and 100. So, this naturally groups into three different clusters.

In [5]:
vectorAssembler = VectorAssembler(inputCols=["col1", "col2", "col3"], outputCol="features")

In [6]:
vcluster_df = vectorAssembler.transform(cluster_df)

In [7]:
vcluster_df.show()

+----+----+----+--------------+
|col1|col2|col3|      features|
+----+----+----+--------------+
|   7|   4|   1| [7.0,4.0,1.0]|
|   7|   7|   9| [7.0,7.0,9.0]|
|   7|   9|   6| [7.0,9.0,6.0]|
|   1|   6|   5| [1.0,6.0,5.0]|
|   6|   7|   7| [6.0,7.0,7.0]|
|   7|   9|   4| [7.0,9.0,4.0]|
|   7|  10|   6|[7.0,10.0,6.0]|
|   7|   8|   2| [7.0,8.0,2.0]|
|   8|   3|   8| [8.0,3.0,8.0]|
|   4|  10|   5|[4.0,10.0,5.0]|
|   7|   4|   5| [7.0,4.0,5.0]|
|   7|   8|   4| [7.0,8.0,4.0]|
|   2|   5|   1| [2.0,5.0,1.0]|
|   2|   6|   2| [2.0,6.0,2.0]|
|   2|   3|   8| [2.0,3.0,8.0]|
|   3|   9|   1| [3.0,9.0,1.0]|
|   4|   2|   9| [4.0,2.0,9.0]|
|   1|   7|   1| [1.0,7.0,1.0]|
|   6|   2|   3| [6.0,2.0,3.0]|
|   4|   1|   9| [4.0,1.0,9.0]|
+----+----+----+--------------+
only showing top 20 rows



In [8]:
kmeans = KMeans().setK(3) #the number of clusters
kmeans = kmeans.setSeed(1) #which determines where the KMeans algorithm starts and this is useful if you're doing testing, to have consistency
kmodel = kmeans.fit(vcluster_df)

What you'll notice here are three points. These points represent the center of three different clusters. One cluster is centered around the points 35, 31, 34, so that's our mid cluster. Another is set around 5, 6, and 5, that's our smallest cluster, our first set of 25 rows, and then the third center point is the center of the cluster for the final set of rows, which are values between 60 and 100. Now in this case, the center point is around to the point 80, 79, 78. So intuitively, this makes sense. KMeans has been able to discover three clusters, and they've centered them as we would expect.

In [9]:
centers = kmodel.clusterCenters() #the centers of these clusters
centers

[array([35.88461538, 31.46153846, 34.42307692]),
 array([80.        , 79.20833333, 78.29166667]),
 array([5.12, 5.84, 4.84])]

## Hierarchical clustering

For large data sets, the hierarchical clustering algorithm, bisecting k-means can be faster. 

In [10]:
from pyspark.ml.clustering import BisectingKMeans

In [11]:
vcluster_df.show()

+----+----+----+--------------+
|col1|col2|col3|      features|
+----+----+----+--------------+
|   7|   4|   1| [7.0,4.0,1.0]|
|   7|   7|   9| [7.0,7.0,9.0]|
|   7|   9|   6| [7.0,9.0,6.0]|
|   1|   6|   5| [1.0,6.0,5.0]|
|   6|   7|   7| [6.0,7.0,7.0]|
|   7|   9|   4| [7.0,9.0,4.0]|
|   7|  10|   6|[7.0,10.0,6.0]|
|   7|   8|   2| [7.0,8.0,2.0]|
|   8|   3|   8| [8.0,3.0,8.0]|
|   4|  10|   5|[4.0,10.0,5.0]|
|   7|   4|   5| [7.0,4.0,5.0]|
|   7|   8|   4| [7.0,8.0,4.0]|
|   2|   5|   1| [2.0,5.0,1.0]|
|   2|   6|   2| [2.0,6.0,2.0]|
|   2|   3|   8| [2.0,3.0,8.0]|
|   3|   9|   1| [3.0,9.0,1.0]|
|   4|   2|   9| [4.0,2.0,9.0]|
|   1|   7|   1| [1.0,7.0,1.0]|
|   6|   2|   3| [6.0,2.0,3.0]|
|   4|   1|   9| [4.0,1.0,9.0]|
+----+----+----+--------------+
only showing top 20 rows



In [12]:
bkmeans = BisectingKMeans().setK(3)
bkmeans = bkmeans.setSeed(1)
bkmodel = bkmeans.fit(vcluster_df)

In [13]:
bkcenters = bkmodel.clusterCenters()
bkcenters

[array([5.12, 5.84, 4.84]),
 array([35.88461538, 31.46153846, 34.42307692]),
 array([80.        , 79.20833333, 78.29166667])]

In [14]:
centers #previous KMeans centers

[array([35.88461538, 31.46153846, 34.42307692]),
 array([80.        , 79.20833333, 78.29166667]),
 array([5.12, 5.84, 4.84])]

Now, in this instance, the bkcenters and centers are the same. That's not guaranteed, though. There may be times where we use k-means and get one set of centers, and use bisecting k-means, and get a different set of centers. While they're not guaranteed to be the same, but they should be pretty close.